In [30]:
import numpy as np
import seaborn as sns

from sklearn.metrics import confusion_matrix

import tensorflow as tf
from tensorflow.keras.layers import Conv2D,Dense,Dropout,MaxPool2D,Flatten
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential,load_model


In [20]:
# training data generator

training_datagen = image.ImageDataGenerator(
    rescale = 1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = image.ImageDataGenerator(
    rescale = 1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [21]:
train_generator = training_datagen.flow_from_directory(
    'Dataset/Train/',
    target_size=(224,224), # most common size, Imagenet is also trained on this size
    batch_size=32,
    class_mode='binary' # 'categorical in case of multiclass classification'
    )

test_generator = test_datagen.flow_from_directory(
    'Dataset/Test/',
    target_size=(224,224), # most common size, Imagenet is also trained on this size
    batch_size=32,
    class_mode='binary' # 'categorical in case of multiclass classification'
    )

Found 312 images belonging to 2 classes.
Found 80 images belonging to 2 classes.


In [16]:
train_generator.class_indices,test_generator.class_indices

({'Covid': 0, 'Normal': 1}, {'Covid': 0, 'Normal': 1})

In [10]:
model = Sequential()

model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(224,224,3)))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPool2D((2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPool2D((2,2)))
model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))

In [11]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 222, 222, 32)      896       
                                                                 
 conv2d_6 (Conv2D)           (None, 220, 220, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 110, 110, 64)      0         
 g2D)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 110, 110, 64)      0         
                                                                 
 conv2d_7 (Conv2D)           (None, 108, 108, 64)      36928     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                 

In [12]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [17]:
history = model.fit_generator(train_generator,steps_per_epoch=8,epochs=10,validation_data=test_generator,validation_steps=2)

/var/folders/d2/7h83324x0yb6w3t360z7mq440000gn/T/ipykernel_11333/2169322337.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator,steps_per_epoch=8,epochs=10,validation_data=test_generator,validation_steps=2)


Epoch 1/10
8/8 [==============================] - 18s 2s/step - loss: 1.5936 - accuracy: 0.5565 - val_loss: 0.6913 - val_accuracy: 0.5156
Epoch 2/10
8/8 [==============================] - 17s 2s/step - loss: 0.6736 - accuracy: 0.6129 - val_loss: 0.6684 - val_accuracy: 0.6562
Epoch 3/10
8/8 [==============================] - 17s 2s/step - loss: 0.5636 - accuracy: 0.7177 - val_loss: 0.4269 - val_accuracy: 0.9375
Epoch 4/10
8/8 [==============================] - 18s 2s/step - loss: 0.4659 - accuracy: 0.8047 - val_loss: 0.4178 - val_accuracy: 0.9531
Epoch 5/10
8/8 [==============================] - 18s 2s/step - loss: 0.3301 - accuracy: 0.8672 - val_loss: 0.2183 - val_accuracy: 0.9375
Epoch 6/10
8/8 [==============================] - 18s 2s/step - loss: 0.3242 - accuracy: 0.9073 - val_loss: 0.3216 - val_accuracy: 0.8750
Epoch 7/10
8/8 [==============================] - 17s 2s/step - loss: 0.2007 - accuracy: 0.9113 - val_loss: 0.1435 - val_accuracy: 0.9688
Epoch 8/10
8/8 [==================

In [18]:
model.save('covid_19.h5')

/usr/local/lib/python3.11/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [23]:
loss,accuracy = model.evaluate(test_generator)

3/3 [==============================] - 2s 613ms/step - loss: 0.1231 - accuracy: 0.9875


In [14]:
# testing model
model = load_model('covid_19.h5')
image_path = 'unseen_test/Covid_positive/1-s2.0-S0929664620300449-gr2_lrg-a.jpg'
img = image.load_img(image_path,target_size=(224,224)) # loading and resizing image
img = image.img_to_array(img)  # converting image to array
img = np.expand_dims(img,axis=0) # expanding dimensions of array
img = img/255
y_pred_normal = mod
y_pred = model.predict(img)

1/1 [==============================] - 0s 67ms/step


In [17]:
np.where(y_pred>0.5,1,0)  # covid positive -> {'Covid': 0, 'Normal': 1} -> model working well

array([[0]])

In [33]:
# test for normal x ray
model = load_model('covid_19.h5')
image_path = 'unseen_test/Normal/IM-0019-0001.jpeg'
img = image.load_img(image_path,target_size=(224,224)) # loading and resizing image
img = image.img_to_array(img)  # converting image to array
img = np.expand_dims(img,axis=0) # expanding dimensions of array
img = img/255
y_pred_normal = model.predict(img)

1/1 [==============================] - 0s 69ms/step


In [35]:
np.where(y_pred_normal>0.5,1,0) # Normal -> {'Covid': 0, 'Normal': 1} -> model working well

array([[1]])